# Creazione degli Indicatori tramite la PCA
Questo codice deve essere eseguito due volte, una con le variabili grezze e una con le variabili shock. Quello che si ottiene sono le componenti principali delle variabili grezze e degli shock.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

Import delle variabili e creazione diszionario.

In [ ]:
# Lista dei paesi e dei nomi dei file. Varia in base a dove e come vengono salvati i dati.
Variables = ["Produzione Industriale", "Spread 10-2", "Inflation", "Unrate", "GDP", "Employment", "Current Account", "Cost of Borrow Corporate"]
file_names = ["prod_shock", "spread_shock", "infl_shock", "unrate_shock", "gdp_shock", "employ_shock", "curracc_shock", "costborrow_"]

# Cartella base del percorso (esempio).
base_folder = "/Users/nome/****/****"

# Dizionario per memorizzare i DataFrame, in questo caso il codice è quello che riguarda gli SHOCK.
Shock = {}

# Ciclo per importare i dataframe di tutte le variabili macroeconomiche.
for variable in Variables:
    for file_name in file_names:
        file_path = f"{base_folder}/{variable}/{file_name}.pkl"
        if os.path.exists(file_path):
            df = pd.read_pickle(file_path)
            df = df.dropna()
            # Salva il DataFrame nel dizionario
            Shock[f"{variable}_{file_name}"] = df
            print(f"Dati caricati per {variable} - {file_name}.")
        else:
            print(f"File non trovato: {file_path}")

Il codice successivo serve per assegnare a tutte le variabili lo stesso indice.

In [ ]:
# 1. Ottieni l'indice di 'Produzione Industriale_shocks_prod'
if 'Produzione Industriale_prod' in Shock:
    indice_comune = Shock['Produzione Industriale_prod'].index
    print(f"Indice comune di 'Produzione Industriale': {indice_comune}")

    # 2. Rimuovi l'indice da tutti i DataFrame tranne quello di 'Produzione Industriale'
    for key in Shock:
        if key != 'Produzione Industriale_prod':
            # Rimuovi l'indice dal DataFrame
            Shock[key] = Shock[key].reset_index(drop=True)
            print(f"Indice rimosso per: {key}")
    
    # 3. Assegna l'indice di 'Produzione Industriale_shocks_prod' a tutti gli altri DataFrame
    for key in Shock:
        if key != 'Produzione Industriale_prod':
            Shock[key].index = indice_comune
            print(f"Indice di 'Produzione Industriale' assegnato a: {key}")
else:
    print("Il DataFrame per 'Produzione Industriale' non è stato trovato.")
# 4. Ora i DataFrame sono allineati con l'indice di 'Produzione Industriale'


Si crea il dataframe finale con all'interno tutte le variabili.

In [ ]:
data = {}

indice_comune = Shock['Produzione Industriale_prod']['Germania'].index

countries = ["Germania", "Francia", "Italia", "Spagna", "Finlandia", "Olanda"]

for country in countries:
    for var, file_name in zip(Variables, file_names):
        column_name = f'{country}_{var.lower().replace(" ", "_")}'  # Es. 'Germania_produzione_industriale'
        
        # Recuperare i dati dal dizionario 'Shock'.
        data[column_name] = Shock[f'{var}_{file_name}'][country].reindex(indice_comune)

df = pd.DataFrame(data)

# Imposta l'indice del DataFrame
df.index = indice_comune

print(df.head())



Si plottano le medie mobili di tutte le variabili per tutte le nazioni (facoltativo)

In [ ]:
num_columns = df.shape[1]

plt.figure(figsize=(15, 20))

# Crea un grafico per ogni gruppo di colonne.
for i in range(8):  
    plt.subplot(8, 1, i + 1) 
    
    columns_to_plot = [j for j in range(i, num_columns, 8)] 
    
    # Calcolare la media mobile a 4 periodi per le colonne selezionate
    for col in columns_to_plot:
        col_name = df.columns[col]  
        plt.plot(df.index, df.iloc[:, col].rolling(window=4).mean(), label=col_name)  

    plt.xlabel('Tempo', fontsize=12)
    plt.ylabel('Valore', fontsize=12)
    plt.grid(True)
    plt.legend(loc='best', fontsize=10)

plt.tight_layout()
plt.show()

## Principal Component Analysis

In [ ]:
# Funzione per calcolare la PCA e ottenere i loadings di un singolo paese.
def calcolare_pca_con_loadings(df, paese, n_components):
    
    df_paese = df.filter(regex=f"^{paese}") # Filtrare i dati per le variabili macro specifiche di ogni paese

    scaler = StandardScaler()
    df_paese_standardizzato = scaler.fit_transform(df_paese)
    
    # Applicare la PCA
    pca = PCA(n_components=n_components)
    pca.fit(df_paese_standardizzato)
    
    return pca

All'interno del ciclo sono commentate le righe di codice in cui sono state invertiti i loadings delle PCA. Questo è stato fatto per i paesi che avevano la componente principale invertita rispetto agli altri paesi.

In [ ]:
# Inizializza una lista per memorizzare i loadings di ogni componente principale per tutti i paesi.
loadings_per_component = {j: [] for j in range(n_components)}

# Calcola i loadings per ogni paese e memorizzarli.
for paese in countries:
    pca_completo = calcolare_pca_con_loadings(df, paese, n_components)
    
    for j in range(n_components):
        loadings = pca_completo.components_[j]
        #if paese == 'Italia' and j in [3]:
        #    loadings = [-x for x in loadings]
        #if paese == 'Spagna' and j in [3]:
        #    loadings = [-x for x in loadings]
        #if paese == 'Finlandia' and j in [0]:
        #    loadings = [-x for x in loadings]
        #if paese == 'Francia' and j in [1]:
        #    loadings = [-x for x in loadings]
        #if paese == 'Olanda' and j in [0]:
        #    loadings = [-x for x in loadings]
        loadings_per_component[j].append(loadings)



Si calcolano e salvano media e deviazione standard dei loadings della stessa componente tra i paesi. Questo passaggio è necessario per la costruzione del grafico loadings PCA all'interno della tesi.

In [ ]:
mean_loadings_per_component = {
    j: np.mean(loadings_per_component[j], axis=0) for j in range(n_components)
}
std_loadings_per_component = {
    j: np.std(loadings_per_component[j], axis=0) for j in range(n_components)
}

# Si salvano i valori per richiamarli all'intenro di un altro notebook in cui viene 
# costruito un unico grafico sia per i loading delle PC grezze che per quelli delle shock.
with open('mean_loadings_per_component_GREZZE.pkl', 'wb') as f:
    pickle.dump(mean_loadings_per_component, f)
with open('std_loadings_per_component_GREZZE.pkl', 'wb') as f:
    pickle.dump(std_loadings_per_component, f)

Viaualizzazione grafica dei loadings.

In [ ]:
plt.figure(figsize=(20, 5))  
plt.suptitle("Media dei Loadings tra paesi ", fontsize=16, fontweight='bold')

for j in range(n_components):
    mean_loadings = mean_loadings_per_component[j]
    std_loadings = std_loadings_per_component[j]
    
    # Crea il subplot per la componente corrente
    ax = plt.subplot(1, n_components, j + 1)  
    
    # Area ombreggiata per deviazione standard
    ax.bar(range(len(mean_loadings)), mean_loadings,
           color=['#4c72b0' if x >= 0 else '#c44e52' for x in mean_loadings])  # Blu per positivi, Rosso per negativi
    
    # Ombreggiatura intorno alla media
    ax.fill_between(range(len(mean_loadings)),
                    np.array(mean_loadings) - np.array(std_loadings),
                    np.array(mean_loadings) + np.array(std_loadings),
                    color='gray', alpha=0.3)
    
    ax.set_xticks(range(len(mean_loadings)))
    ax.set_xticklabels(Variables, rotation=45, ha="right", fontsize=10)
    
    ax.set_title(f'PC{j+1} Grezze', fontsize=14, fontweight='bold')
    ax.set_ylabel('Loading Medio', fontsize=12)
    
    ax.set_ylim(-1, 1) 
    
    ax.grid(True, linestyle='-', linewidth=0.5, color='gray', alpha=0.5)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.savefig('media_loadings_pca_GREZZE_INVERTITE.png', dpi=300, bbox_inches='tight')

plt.show()


## Salvataggio degli Indicatori (Componenti Principali)

In [ ]:
# Creazione dei DataFrame per PC1, PC2, PC3 e PC4

df_pc1 = pd.DataFrame({
    'Germania': germania_pca[:, 0],

    'Francia': francia_pca[:, 0],
    'Italia': italia_pca[:, 0],
    'Spagna': spagna_pca[:, 0],
    'Finlandia': finlandia_pca[:, 0],
    'Olanda': olanda_pca[:, 0]
}, index=df.index)

df_pc2 = pd.DataFrame({
    'Germania': germania_pca[:, 1],
    'Francia': francia_pca[:, 1],
    'Italia': italia_pca[:, 1],
    'Spagna': spagna_pca[:, 1],
    'Finlandia': finlandia_pca[:, 1],
    'Olanda': olanda_pca[:, 1]
}, index=df.index)

df_pc3 = pd.DataFrame({
    'Germania': germania_pca[:, 2],
    'Francia': francia_pca[:, 2],
    'Italia': italia_pca[:, 2],
    'Spagna': spagna_pca[:, 2],
    'Finlandia': finlandia_pca[:, 2],
    'Olanda': olanda_pca[:, 2]
}, index=df.index)

df_pc4 = pd.DataFrame({
    'Germania': germania_pca[:, 3],
    'Francia': francia_pca[:, 3],
    'Italia': italia_pca[:, 3],
    'Spagna': spagna_pca[:, 3],
    'Finlandia': finlandia_pca[:, 3],
    'Olanda': olanda_pca[:, 3]
}, index=df.index)

In [ ]:
#Si invertono i valori delle PCA con loadings invertiti rispetto agli altri paesi.
df_pc1['Olanda'] = df_pc1['Olanda'] * -1
df_pc2['Francia'] = df_pc2['Francia'] * -1
df_pc4['Germania'] = df_pc4['Germania'] * -1
df_pc4['Francia'] = df_pc4['Francia'] * -1
df_pc4['Italia'] = df_pc4['Italia'] * -1
df_pc4['Finlandia'] = df_pc4['Finlandia'] * -1

df_pc1.to_pickle('pc1_grezze_all_nations.pkl')
df_pc2.to_pickle('pc2_grezze_all_nations.pkl')
df_pc3.to_pickle('pc3_grezze_all_nations.pkl')
df_pc4.to_pickle('pc4_grezze_all_nations.pkl')